In [72]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import lightgbm as lgb
import seaborn as sns
from sklearn.preprocessing import StandardScaler


data = pd.read_csv('data/train.csv')
data['Gender'] = data['Gender'].map({'Male': 0, 'Female': 1})

data['TP_Tbil_ratio'] = data['TP']/data['T_Bil'] 
data['ALTGPT_TP_ratio'] = data['ALT_GPT']/data['TP']
 

data['Age_Tbil_ratio'] = data['Age']/data['T_Bil'] 
data['Age_Dbil_ratio'] = data['Age']/data['D_Bil']
data['Age_ALP_ratio'] = data['Age']/data['ALP']
data['Age_ALTGPT_ratio'] = data['Age']/data['ALT_GPT']
data['Age_ASTGOT_ratio'] = data['Age']/data['AST_GOT']
data['Age_TP_ratio'] = data['Age']/data['TP']
data['Age_ALb_ratio'] = data['Age']/data['Alb']
data['Age_AG_ratio'] = data['Age']/data['AG_ratio']
'''
data['ALTgpt_ASTgot_diff'] = data['ALT_GPT'] - data['AST_GOT'] 
data['Tbil_Dbil_diff'] = data['T_Bil'] - data['D_Bil'] 
data['ALTgpt_ASTgot_prod'] = data['ALT_GPT'] * data['AST_GOT'] 
data['Tbil_Dbil_prod'] = data['T_Bil'] * data['D_Bil'] 
'''

"\ndata['ALTgpt_ASTgot_diff'] = data['ALT_GPT'] - data['AST_GOT'] \ndata['Tbil_Dbil_diff'] = data['T_Bil'] - data['D_Bil'] \ndata['ALTgpt_ASTgot_prod'] = data['ALT_GPT'] * data['AST_GOT'] \ndata['Tbil_Dbil_prod'] = data['T_Bil'] * data['D_Bil'] \n"

In [73]:
z_scores = (data - data.mean()) / data.std()

# Zスコアがしきい値を超える行を抽出
threshold = 3
outliers = data[(z_scores > threshold).any(axis=1)]
# 外れ値を含む行を削除
cleaned_data = data.drop(outliers.index)

# インデックスをリセット（オプション）
cleaned_data.reset_index(drop=True, inplace=True)


In [ ]:
columns_to_plot =  data.columns
for column in columns_to_plot:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x='disease', y=column, data=cleaned_data)
    plt.title(f'Box Plot of {column} by Disease')
    plt.xlabel('Disease (0: No, 1: Yes)')
    plt.ylabel(column)
    plt.show()

In [44]:
cleaned_data.drop('disease', axis=1).head

<bound method NDFrame.head of      Age  Gender     T_Bil     D_Bil         ALP    ALT_GPT     AST_GOT  \
0     59       0  0.786886  0.150498  220.178691  13.467617   21.729246   
1     69       0  1.003987  0.195625  221.218413  51.033462   64.752323   
2     65       0  0.906822  0.214173  369.278160  34.347597   54.510085   
3     22       1  1.734959  0.197706  222.782025  20.572891  170.010177   
4     44       0  0.778259  0.078457  211.647469  19.173386   97.312206   
..   ...     ...       ...       ...         ...        ...         ...   
765   25       0  0.767450  0.186467  215.656126  13.963269   20.160997   
766   57       0  0.867946  0.156997  213.810275  14.564305   20.122535   
767   50       0  0.826785  0.138898  230.340081  40.495102   49.620857   
768   32       1  0.761972  0.137969  214.170393  18.048321   54.447774   
769   64       0  0.762713  0.218435  214.720510  15.885571   24.789606   

           TP       Alb  AG_ratio  TP_Tbill_ratio  ALTGPT_TP_ratio  \

In [74]:
scaler = StandardScaler()
cleaned_data=cleaned_data.drop('disease', axis=1)
col = cleaned_data.columns
cleaned_data = pd.DataFrame(scaler.fit_transform(cleaned_data))
cleaned_data.columns=col
cleaned_data['disease'] = data['disease']

print(len(cleaned_data))

760


In [75]:
from scipy.stats import ttest_ind

for column in cleaned_data.columns:
    group_0 = cleaned_data[cleaned_data['disease'] == 0][column]
    group_1 = cleaned_data[cleaned_data['disease'] == 1][column]

    t_stat, p_value = ttest_ind(group_0, group_1)
    print(f'{column}: t-statistic = {t_stat}, p-value = {p_value}')

Age: t-statistic = -0.45826282450798855, p-value = 0.6468949775308018
Gender: t-statistic = -0.17176314376645732, p-value = 0.8636695864917081
T_Bil: t-statistic = -2.6190397088418726, p-value = 0.00899399288661654
D_Bil: t-statistic = -2.5714965815111683, p-value = 0.010315524872487113
ALP: t-statistic = 0.3643372810469331, p-value = 0.7157077749678227
ALT_GPT: t-statistic = -1.3045185141607414, p-value = 0.1924527868898858
AST_GOT: t-statistic = -1.4641642468998675, p-value = 0.14356366090469694
TP: t-statistic = 1.7898182129068205, p-value = 0.07388221016925603
Alb: t-statistic = 1.8179739372738182, p-value = 0.0694627228648942
AG_ratio: t-statistic = 1.2271887421798562, p-value = 0.22013266053891045
TP_Tbill_ratio: t-statistic = 3.482696884445702, p-value = 0.0005248512633413977
ALTGPT_TP_ratio: t-statistic = -1.4236797272203927, p-value = 0.1549506904488592
Age_Tbill_ratio: t-statistic = 2.1052239142105598, p-value = 0.03560030061019423
Age_Dbill_ratio: t-statistic = 1.31548950517

/tmp/ipykernel_362/2664877873.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat, p_value = ttest_ind(group_0, group_1)


In [167]:
df_disease = cleaned_data[cleaned_data['disease']==1]
df_good = cleaned_data[cleaned_data['disease']==0]
df_disease.describe()

,Age,Gender,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease,Age_Tbill_ratio
count,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.0,316.000000
mean,47.379747,0.142405,1.836804,0.601846,277.652580,30.126455,55.853708,6.972942,3.424596,1.069099,1.0,39.904316
std,16.396518,0.350019,1.949183,0.809571,117.244314,35.355598,40.801235,0.810561,0.560269,0.233650,0.0,22.767939
min,16.000000,0.000000,0.642805,0.074207,185.491189,8.570383,11.601122,4.971703,2.180884,0.627521,1.0,2.750665
25%,32.000000,0.000000,0.845703,0.177288,220.753969,17.380286,26.699817,6.760085,3.122608,0.969728,1.0,23.014476
50%,48.000000,0.000000,0.985981,0.230953,226.866746,21.922399,52.677577,6.895087,3.339587,1.019172,1.0,36.717823
75%,64.000000,0.000000,1.820910,0.665063,236.004914,28.959043,62.112826,7.507488,3.699607,1.256375,1.0,57.797328
max,75.000000,1.000000,8.937923,4.520551,669.527004,370.984154,368.629116,8.739000,5.014720,1.813936,1.0,101.119249


In [168]:
df_good.describe()

,Age,Gender,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease,Age_Tbill_ratio
count,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.0,470.000000
mean,46.202128,0.210638,0.824109,0.184432,234.029770,15.773021,28.919373,7.083418,3.604229,1.207416,0.0,57.574433
std,16.283849,0.408196,0.174403,0.115674,67.108673,7.015401,19.166554,0.859678,0.556548,0.200991,0.0,21.932506
min,10.000000,0.000000,0.585961,0.034861,163.261838,3.924518,11.283497,4.858679,2.295068,0.732968,0.0,12.680416
25%,32.000000,0.000000,0.745857,0.120325,211.841497,12.546847,20.261433,6.713348,3.163718,1.029670,0.0,38.025153
50%,48.000000,0.000000,0.796513,0.166267,215.761897,14.825259,22.265808,7.304111,3.630569,1.281130,0.0,57.025042
75%,61.000000,0.000000,0.836139,0.200841,221.374780,17.014089,27.942651,7.538401,4.059950,1.294678,0.0,75.829454
max,78.000000,1.000000,1.734959,0.806529,657.616053,86.911871,170.010177,8.733841,5.016970,1.821496,0.0,114.370398


In [169]:
#data.describe()

In [170]:
#df_disease = data[data['disease']==1]
#df_good = data[data['disease']==0]
#df_disease.describe()

In [171]:
#df_good.describe()